In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GroupKFold, GridSearchCV, GroupShuffleSplit
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder

In [ ]:
# Load data from external CSV file. E.g., 

df = pd.read_csv(...)
print (df)

In [ ]:
# ElasticNet modeling

In [ ]:
# X is the input of predictor variables. Dummy names shown below instead of the real variable names.
# y is the outcome variable
# groups is the array indicating the hierarchical structure (e.g., state)

X = df[['var1','var2','etc']]  
y = df[['outcome']]
groups = df[['STATE']]

# Convert groups to numeric representation using LabelEncoder
le = LabelEncoder()
groups_numeric = le.fit_transform(groups)

# Create a GroupShuffleSplit object to perform the train-test split
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=237)

# Perform the train-test split while maintaining the hierarchical/blocked data structure
train_index, test_index = next(gss.split(X, y, groups_numeric))

if isinstance(X, pd.DataFrame):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
else:
    X_train, X_test = X[train_index], X[test_index]

y_train, y_test = y.iloc[train_index], y.iloc[test_index]
groups_train, groups_test = groups_numeric[train_index], groups_numeric[test_index]

# Create a GroupKFold object to handle hierarchical data
gkf = GroupKFold(n_splits=5)

# Define the parameter grid for hyperparameter tuning. E.g.,
param_grid = {'alpha': [0.001, 0.01, 0.05, 0.1, 1, 10, 100], 'l1_ratio': [0.001, 0.01, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.99, 1.0]}

# Create an ElasticNet regression model
elastic_net = ElasticNet(max_iter=10000, tol=1e-3)

# Perform grid search with nested cross-validation for hierarchical data
grid_search = GridSearchCV(estimator=elastic_net, param_grid=param_grid, cv=gkf, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train, groups=groups_train)

# Get the best model and its parameters
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

print("Best parameters: ", best_params)

# Evaluate the model using nested cross-validation
cv_scores = -grid_search.cv_results_['mean_test_score']
mse_scores = np.sqrt(cv_scores)

print("Nested cross-validation MSE scores: ", mse_scores)

# Evaluate the best model on the unseen outer loop test set
best_model.fit(X_train, y_train)
y_pred_train = best_model.predict(X_train)
y_pred_test = best_model.predict(X_test)

# Calculate MSE for the test set
test_mse = mean_squared_error(y_test, y_pred_test, squared=False)
print("Test set MSE: ", test_mse)

# Calculate RMSE for the test set
test_rmse = np.sqrt(test_mse)
print("Test set RMSE: ", test_rmse)

# Train the model on each validation fold separately and obtain feature importances
importances_list = []
for train_index, val_index in gkf.split(X_train, y_train, groups_train):
    X_fold_train, X_fold_val = X_train.iloc[train_index], X_train.iloc[val_index]
    y_fold_train, y_fold_val = y_train.iloc[train_index], y_train.iloc[val_index]
    
    fold_model = ElasticNet(**best_params)
    fold_model.fit(X_fold_train, y_fold_train)
    
    fold_importances = fold_model.coef_
    importances_list.append(fold_importances)

# Calculate the average feature importances across folds
importances_mean = np.mean(importances_list, axis=0)

# Create a dataframe to store the average feature importances
if isinstance(X, pd.DataFrame):
    feature_importances = pd.DataFrame({'Feature': X.columns, 'Importance': np.abs(importances_mean)})
else:
    feature_importances = pd.DataFrame({'Feature': range(X.shape[1]), 'Importance': np.abs(importances_mean)})

# Sort the feature importances in descending order
feature_importances = feature_importances.sort_values(by='Importance', ascending=False)

# Display the top 10 most important predictors and their absolute values
print("Top 10 Most Important Predictors (Average across Folds):")
print(feature_importances.head(10))

#Confirm shape of data
print("Shape of y:", y.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)
print("Shape of y_fold_train:", y_fold_train.shape)